In [1]:
# allow import of parent modules - worked for me, may be dumb
import sys  
sys.path.insert(0, '../')

# imports 
from preprocessing.preprocessing import get_preprocessed_brfss_train_test_split_one_hot_encoded, get_preprocessed_brfss_dataset, download_brfss_dataset,get_preprocessed_brfss_train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

import matplotlib.pyplot as plt

import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [3]:
use_mps = False
use_cuda = False

device = torch.device("cpu")

if torch.has_mps and use_mps:
    device = torch.device("mps")
    print("Torch mps activated")

if torch.has_cuda and use_cuda:
    device = torch.device("cuda")
    print("Torch cuda activated")

In [2]:

#download_brfss_dataset("donatomonti", "8d7c0d3df66d7edf2ff8842a6caa1a5c")
#train_dataset, target = get_preprocessed_brfss_dataset()

train_dataset, test_dataset, train_target, test_target = get_preprocessed_brfss_train_test_split()

In [3]:
train_dataset = train_dataset.head(5000)
train_target = train_target.head(5000)
test_dataset = test_dataset.head(5000)
test_target = test_target.head(5000)

print(len(train_dataset))
print(len(train_target))
print(len(test_dataset))
print(len(test_target))



5000
5000
5000
5000


In [4]:

# create support vector machine estimator
# Radial Basis Function Kernel -> a popular kernel function commonly used in support vector machine classification. RBF can map an input space in infinite dimensional space.
svm = SVC(kernel="rbf")
#Train the model using the training sets
svm.fit(train_dataset, train_target.values.ravel())

#Predict the response for test dataset
test_predictions = svm.predict(test_dataset)

print("Accuracy:",metrics.accuracy_score(test_target, test_predictions))

Accuracy: 0.8476


In [5]:
# descriptions from https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python


# define parameter grid for SVM
# C: Regularization parameter in python's Scikit-learn C parameter used to maintain regularization. The misclassification or error term tells the SVM optimization how much error is bearable.
# gamma: A lower value of Gamma will loosely fit the training dataset, whereas a higher value of gamma will exactly fit the training dataset, which causes over-fitting. 

parameter_grid = {"C" : [1, 10, 100, 1000], "gamma" : [0.001, 0.01, 0.1, 1]}

# create grid search for parameter optimization and model selection
grid_search = GridSearchCV(svm, parameter_grid, scoring="accuracy", cv=5)

acc_cv = cross_val_score(grid_search,train_dataset, train_target.values.ravel(), cv=5, scoring="accuracy")
print(acc_cv)


/Users/donatomonti/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/donatomonti/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 